In [ ]:
import sys
# import modules in other directories
sys.path.append('../Utils')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import trax_util as trx
import ibtracs as IBT
import TRMM_util as trmm
import Precip_util as prc
import PlotUtil as Pu

import importlib

importlib.reload ( trmm )
importlib.reload ( prc )
importlib.reload ( Pu )



In [ ]:
read_in_npsave=True
if (read_in_npsave==True):
    npSaveFile = '/glade/work/juliob/NumPySaves/MeanTC_xTSprecip.npz'
    SaveData = np.load( npSaveFile )
    TRMM_tcprec_grand_av = SaveData['TRMM_tcprec_grand_av']
    PD_tcprec_grand_av = SaveData['PD_tcprec_grand_av']
    SST7_tcprec_grand_av = SaveData['SST7_tcprec_grand_av']
    lono=SaveData['lono']
    lato=SaveData['lato']
    lonc=SaveData['lonc']
    latc=SaveData['latc']
else:
    ##
    year0,year1 = 1998,2012
    nyrs = 0
    prec_grand_av = 0.
    for Yr in np.arange( start=year0 , stop=year1+1 ):
        yA = str( Yr ).zfill(4)
        ftrmm = "/glade/p/cgd/amp/juliob/TRMM/3B42_TCs/timeseries/3B42_3hrly_TCPRECT." + yA + ".nc"
        ds=xr.open_dataset( ftrmm )
        prec_av = trmm.timemean( ds.TCPRECT )
        prec_grand_av = prec_grand_av + prec_av
        print(f"File = {ftrmm} ")
        nyrs=nyrs+1

    TRMM_tcprec_grand_av = 24.*prec_grand_av / nyrs
    lono=ds.lon.values
    lato=ds.lat.values

    ##
    # CAM part 
    # input and output Directories
    drc_o='/glade/campaign/cgd/amp/juliob/TC-cesm1/precip/'

    nyrs = 0
    prec_grand_av = 0.
    for e in np.arange( start=1,stop=4 ):
        BaseName = trx.pdfname(ens=e,justBaseName=True) 
        print( f"BaseName is  {BaseName} " )
        basename_o = BaseName + 'cam.h4.TCPRECT.'
        for Yr in np.arange( start=year0 , stop=year1+1 ):
            yA = str( Yr ).zfill(4)
            fname =drc_o+basename_o+yA+'010100Z-'+yA+'123121Z.nc'
            ds=xr.open_dataset( fname )
            prec_av = trmm.timemean( ds.TCPRECT )
            prec_grand_av = prec_grand_av + prec_av
            print(f"File = {fname} ")
            nyrs=nyrs+1

    PD_tcprec_grand_av = 1000. * 86400. * prec_grand_av / nyrs
    lonc=ds.lon.values
    latc=ds.lat.values


    ##
    # CAM part 
    BaseName = trx.rcp85fname(sst='sst7',justBaseName=True) 
    print( f"BaseName is  {BaseName} " )
    basename_o = BaseName + 'cam.h4.TCPRECT.'
    # input and output Directories
    drc_o='/glade/campaign/cgd/amp/juliob/TC-cesm1/precip/'


    year0,year1 = 2070,2089 #2012
    nyrs = 0
    prec_grand_av = 0.
    for Yr in np.arange( start=year0 , stop=year1+1 ):
        yA = str( Yr ).zfill(4)
        fname =drc_o+basename_o+yA+'010100Z-'+yA+'123121Z.nc'
        ds=xr.open_dataset( fname )
        prec_av = trmm.timemean( ds.TCPRECT )
        prec_grand_av = prec_grand_av + prec_av
        print(f"File = {fname} ")
        nyrs=nyrs+1

    SST7_tcprec_grand_av = 1000. * 86400. * prec_grand_av / nyrs
    lonc=ds.lon.values
    latc=ds.lat.values


In [ ]:
cmapN='gist_rainbow'
preclv=np.array( [0.01, 0.05, 0.07, 0.1, 0.2, 0.5, 1.] )
cmap=Pu.step_cmap( levels=preclv , cmapN=cmapN )

projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,10))
lwdth=1

n=0
nxplo,nyplo=2,1

n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
print(Axes)
ax1 = fig.add_axes( Axes , projection=projPC )

ax1.coastlines(resolution='110m',color='black',linewidth=lwdth)
co = ax1.contourf( lono, lato, TRMM_tcprec_grand_av ,levels=preclv,cmap=cmap,transform=data_crs )
#plt.colorbar( co , ax=ax1 )
#crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0., yshift=0.1) )
#cbar=fig.colorbar( co, cax=crax, ax=ax1 )
#cbar.set_label( 'mm d$^{-1}$', size=16 )
ax1.set_global()
ax1.spines['geo'].set_linewidth(lwdth)  # change as needed
fig.text( Axes[0]-.02,Axes[1]+Axes[3]-.2, 'A)' , size=24)
ax1.set_title( "TRMM/IBTrACS: Storm precipitation (TS+)" ,size=24)

#######################
n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
print(Axes)
ax1 = fig.add_axes( Axes , projection=projPC )


ax1.coastlines(resolution='110m',color='black',linewidth=lwdth)
co = ax1.contourf( lonc, latc, PD_tcprec_grand_av ,levels=preclv,cmap=cmap,transform=data_crs )
crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0.05, yshift=0.3) )
cbar=fig.colorbar( co, cax=crax ,ax=ax1 )
cbar.set_label( 'mm d$^{-1}$', size=16 )
ax1.set_global()
ax1.spines['geo'].set_linewidth(lwdth)  # change as needed
fig.text( Axes[0]-.02,Axes[1]+Axes[3]-.2, 'B)' , size=24)
ax1.set_title( "CAM PD ensemble-mean: Storm precipitation (TS+)" ,size=24)

Pu.save_link(png_filename = "TCprecipMean-PD.png")



In [ ]:
cmapN='gist_rainbow'
preclv=np.array( [0.01, 0.05, 0.07, 0.1, 0.2, 0.5, 1.] )
cmap=Pu.step_cmap( levels=preclv , cmapN=cmapN )

projPC = ccrs.Robinson(central_longitude=0.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,18))

n=0
nxplo,nyplo=1,2

n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
print(Axes)
ax1 = fig.add_axes( Axes , projection=projPC )

ax1.coastlines(resolution='110m',color='black',linewidth=4)
co = ax1.contourf( lono, lato, TRMM_tcprec_grand_av ,levels=preclv,cmap=cmap,transform=data_crs )
#plt.colorbar( co , ax=ax1 )
crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0., yshift=0.1) )
cbar=fig.colorbar( co, cax=crax, ax=ax1 )
cbar.set_label( 'mm d$^{-1}$', size=16 )
ax1.set_global()
ax1.spines['geo'].set_linewidth(4)  # change as needed
fig.text( Axes[0],Axes[1]+Axes[3], 'A)' , size=24)
ax1.set_title( "TRMM/IBTrACS: Storm precipitation (TS+)" ,size=24)

#######################
n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo)
print(Axes)
ax1 = fig.add_axes( Axes , projection=projPC )


ax1.coastlines(resolution='110m',color='black',linewidth=4)
co = ax1.contourf( lonc, latc, PD_tcprec_grand_av ,levels=preclv,cmap=cmap,transform=data_crs )
crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0., yshift=0.1) )
cbar=fig.colorbar( co, cax=crax ,ax=ax1 )
cbar.set_label( 'mm d$^{-1}$', size=16 )
ax1.set_global()
ax1.spines['geo'].set_linewidth(4)  # change as needed
fig.text( Axes[0],Axes[1]+Axes[3], 'B)' , size=24)
ax1.set_title( "CAM PD ensemble-mean: Storm precipitation (TS+)" ,size=24)


In [ ]:
#######################
#ax1 = fig.add_subplot(3, 1, 2 , projection=projPC )
print(Axes)
ax1 = fig.add_axes( Axes , projection=projPC )


ax1.coastlines(resolution='110m',color='black',linewidth=4)
co = ax1.contourf( lonc, latc, SST7_tcprec_grand_av ,levels=preclv,cmap=cmap,transform=data_crs )
crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0., yshift=.1) )
cbar=fig.colorbar( co, cax=crax ,ax=ax1 )
ax1.set_global()
ax1.spines['geo'].set_linewidth(4)  # change as needed
fig.text( Axes[0],Axes[1]+Axes[3], 'C)' , size=24)
#ax1.text( Axes[0],Axes[1]+Axes[3], 'B)', transform=ax1.transAxes )

In [ ]:
#######################
ax1 = fig.add_subplot(3, 1, 3 , projection=projPC )

ax1.coastlines(resolution='110m',color='black',linewidth=4)
co = ax1.contourf( lonc, latc, CESM_SST2_tcprec_grand_av ,levels=preclv,cmap='cool_r',transform=data_crs )
"""
for ns in np.arange( nstorms ):
    if (trk.year[ns,0]==2004):
        #zoo=ax1.plot( trk.lon[ns,:], trk.lat[ns,:], transform=data_crs  )
        pass
#ax1.add_feature(cfeature.LAND,alpha=.9)
"""
plt.colorbar( co , ax=ax1 )
ax1.set_global()
ax1.spines['geo'].set_linewidth(4)  # change as needed

In [ ]:

trk=IBT.readtrx()


In [ ]:
nstorms,ntraxtime=np.shape( trk.lon )

In [ ]:
print(np.shape(prec_grand_X))

In [ ]:
plt.contourf(prec_grand_X[3,:,:],levels=preclv,cmap='cool_r' )
plt.xlim(1010,1020)
plt.ylim(250,260)
print(prec_grand_X[3,255,1019])

In [ ]:
poo=ds.TCPRECT[:,255,1019].values*24
plt.plot( poo ,'+')
plt.ylim( 500,1000)
plt.xlim(1500,1600)
